UM MSBA - BGEN632

# Week 7: Data Manipulation

The purpose of this tutorial is to familiarize you with how to access data, import data, and manipulate that data into usable formats. This requires knowledge of data frames.

To begin, we will go over the concept of a data frame. Then, we will dive in to how to import data into a data frame object, access the contents of the data frame by index values, sort the data, and several other functions and methods.

---

## Notebook Setup

To setup a notebook, we should first load required modules and set the working directory.

Following best practice, we will place import statements for the libraries/modules/packages used in this tutorial at the beginning of the notebook. You will need to download and install the following packages using your preferred package manager if you have not already done so:

* `pandas`
* `numpy`
* `scikit-learn`

We will also cover the `os` module in the tutorial, but this is included in Python's standard library so you do not need to complete a separate installation. 

### Import Modules

Run the code cell below to load required modules. Note that we are only importing the `KFold` class from `sklearn.model_selection` in scikit-learn.

In [1]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold

### Set Working Directory

Your working directory is the filepath where files are read into and saved out of Jupyter/Python. In other words, it is the location on your machine that Jupyter and Python assume is the starting place for all paths that you try to access or construct. The default working directory in a Jupyter Notebook file is the directory in which it is saved.

To determine the current directory, we can use the `getcwd()` function from the `os` module:

In [2]:
os.getcwd()  # get current working directory

'c:\\Users\\winst\\OneDrive\\Desktop\\Python 2\\Week7\\week7labs'

Consider a hypothetical situation where you have a designated folder for your data science work. This folder contains your data and is also the location where you would like to save analysis results and visualizations. What to do? You could change your working directory to that location by using `os.chdir()` like so:

```Python
os.chdir()   # add your desired file path within the parentheses of this function to change the directory
os.getcwd()  # then confirm the change by checking the current working directory
```

For this tutorial, your working directory will be set to the location of your Week 7 materials (i.e., the file path for your local copy of the Week 7 GitHub repo). Specifically, we will work in the data folder. Note that the working directory convention differs for Windows vs. Mac and Linux. My machine is a Mac so the code in my notebook looks like this:

```Python
os.chdir("/Users/obn/Documents/GitHub/UM-BGEN632/week7labs/data")
```

Add your desired file path within the parentheses of the `os.chdir()` function in the code cell below and then run it:

In [9]:
os.chdir("c:\\Users\\winst\\OneDrive\\Desktop\\Python 2\\Week7\\week7labs\\data")   # change the directory
os.getcwd()  # confirm the change 

'c:\\Users\\winst\\OneDrive\\Desktop\\Python 2\\Week7\\week7labs\\data'

#### A Quick Aside: Defining a Path

You can also define absolute or relative filepaths and use those filepaths as needed. A relative path is the path that is relative to the working directory location on your machine. An absolute path is a path that contains the entire path to the file or directory that you need to access. This path will begin at the home directory of your machine and will end with the file or directory that you wish to access. 

Absolute paths ensure that Python can find the exact file on your machine. However, machines can have a different path constructions, depending on the operating system, and contain usernames that are unique to that specific machine. There are ways to overcome this issue and others associated with finding files on different machines. The `os.path.join()` and `os.path.exists()` functions are particularly useful for finding files on different machines. [Check out this resource from the UVA Research Computing Portal to learn more](https://learning.rc.virginia.edu/courses/python-introduction/files/). 

---

## Data Frame Overview

Okay, let's talk about data. We can organize data into a data matrix (i.e., a table) where each row represents a thing observed and each column represents a variable. We might also refer to this as *tabular* data as the data is organized and displayed as a table. In the example below, some employee data for a fictional company with 63 employees is presented in a table. 

| Employee_ID | Tenure_Months | Leadership | Title | 
|:----|:---- |:---- |:---- |
| e1 | 45 | $$True$$ | Project Manager |
| e2 | 2 | $$False$$ | Junior Analyst |
| e3 | 13 | $$False$$ | Adminstrative Assistant |
| . | . | . | . |
| . | . | . | . |
| e63 | 19 | $$True$$ | Senior Engineer |


### pandas DataFrame

In Python, we can use the Python Data Analysis Library (pandas) to work with tabular data. This library provides the tools and data structures we need to effectively work with data. To do so, we need to load pandas in our notebook. Recall that we can simplify its name based on standard convention as we will use it over and over again.

```Python
import pandas as pd
```

In pandas, the primary data structure is a `DataFrame`. This is what we will use to represent and interact with our data. For those of you who have experience with *R Statistical Software*, you are likely already familiar with the concept of a data frame. 

So, what is it exactly? A pandas `DataFrame` is a 2-dimensional data structure. It can store data of different types (including characters, integers, floating point values, categorical, and more) in columns. Stated another way, it is a table with rows and columns. 

Let's return to the fictional employee data example provided above to highlight the qualities of a DataFrame:

* Each row is a single *observation*: an employee.
  * That means that the data in a row are *related* to each other.
  * In this case, the values in a row all correspond to a single employee.
* Each column is a single *variable*: employee ID, tenure, leadership, title.
  * Columns are *named* (e.g., 'Tenure_Months' is the name of the second column).
  * All columns within a DataFrame contain the same number of items and rows (63 in this example).
  * Each variable has a specific data type (string, integer, boolean, categorical in this example, respectively).
    * Note that while different columns may have different types, *each column must contain only one type of data*!
    * In Python, you may encounter a `None` object which represents the absence of a value (or a null value) in a column.
    * You may also encounter a `NaN` (Not a Number) object which represents undefined or nonsensical numerical values.
    * [Here's a Medium article describing the difference between None and NaN](https://medium.com/data-and-beyond/none-nan-null-and-zero-in-python-ac326cfb73a2).

These details ultimately represent metadata, or data about data:

* Column names
* Column data types
* Length of columns | Number of rows
* Column order
* Row order


#### Data Frame Differences: R vs. Python

Data frames are built into base R. This means that the functions, commands, and other operations within R work very well with data frames. In contrast, data frames are provided by an external package (pandas) in Python. This means that data frames are not as well integrated in Python as in R. To help overcome this limitation, we can rely on data science libraries like NumPy and scikit-learn. Keep in mind that software updates may break the functionality of data frames. You may need to revert back to an older version of a library/module/package to regain functionality. I tend to prefer R for data manipulation, analysis, and visualization due to these and  other reasons. 


### From Spreadsheet to DataFrame

If the fictional employee data table provided above were stored in a spreadsheet, we could import it in Python as a pandas DataFrame. For example, if the data were contained within a CSV file named 'employee_data.csv', we could import the data by using the pandas `read_csv()` function and store the data like so:

```Python
employees = pd.read_csv("employee_data.csv")
```

This example also shows the use of our defined name `pd` for `pandas` functions! Also, while this example is based on a CSV file, pandas can support many file formats or data sources (csv, excel, sql, json, parquet, …), each of them with the prefix read_*:


```Python
pd.read_csv("our_data.csv")
pd.read_excel("our_data.excel")
pd.read_sql("our_data.sql")
pd.read_json("our_data.json")
# etc.
```

### Load Data

For this tutorial, we will start by working with the pine tree data in the Loblolly.csv file. [This dataset is from the R Datasets package](https://www.rdocumentation.org/packages/datasets/versions/3.6.2/topics/Loblolly) and is provided within the data folder of this week's repo. 

Run the code cell below to load and store the Loblolly dataset.

In [10]:
loblolly = pd.read_csv("Loblolly.csv")

### Inspect Data 

After reading in data, you should **always** check it! You should never assume data is perfect and ready for analysis. You should inspect data for errors, inconsistencies, and missing values. You should also verify data types because this information will influence the accuracy and appropriateness of various operations applied to the data.

Familiarizing yourself with the data in a DataFrame may entail selecting specific columns or rows, sorting data, and selecting data based on conditions. In this section, we will go over some approaches for inspecting data.

#### Heads and Tails

To quickly inspect the data, we can use the `head(n)` or `tail(n)` functions, where `n` is the number of lines that should be returned. Specifically, we can use `head(n)` to view the first `n` lines in a DataFrame and we can use `tail(n)` to view the last `n` lines in a DataFrame. Alternatively, you can display the first and last five rows of the DataFrame with just the DataFrame name.

```Python
loblolly.head()    # display the first five rows of the DataFrame (the default)
loblolly.head(10)  # display the first 10 rows of the DataFrame
loblolly.head(50)  # display the first 50 rows of the DataFrame

loblolly.tail()    # display the last five rows of the DataFrame (the default)
loblolly.tail(10)  # display the last 10 rows of the DataFrame
loblolly.tail(50)  # display the last 50 rows of the DataFrame

loblolly # display the first and last five rows of the DataFrame
```

Run the code cells below to inspect the data. Replace the `n` in `head()` and `tail()` with a number to specify how many rows should be displayed. 

In [11]:
loblolly  # display the first and last five rows of the DataFrame

,height,age,Seed
0,4.51,3,301
1,4.55,3,303
2,4.79,3,305
3,3.91,3,307
4,4.81,3,309
...,...,...,...
79,61.62,25,323
80,58.49,25,325
81,56.81,25,327
82,56.43,25,329


In [12]:
loblolly.head(7)  # display the first n rows of the DataFrame (replace the n with desired number)

,height,age,Seed
0,4.51,3,301
1,4.55,3,303
2,4.79,3,305
3,3.91,3,307
4,4.81,3,309
5,3.88,3,311
6,4.32,3,315


In [13]:
loblolly.tail(7)  # display the last n rows of the DataFrame (replace the n with desired number)

,height,age,Seed
77,60.69,25,319
78,60.28,25,321
79,61.62,25,323
80,58.49,25,325
81,56.81,25,327
82,56.43,25,329
83,59.49,25,331


#### Columns

To lookup the names of the columns within a DataFrame, we can use the columns attribute. 

Run the code cell below to view the names of columns in the `loblolly` DataFrame.

In [14]:
loblolly.columns

Index(['height', 'age', 'Seed'], dtype='object')

For a DataFrame object, we use a period to reference columns in Python (in R, we use the $ symbol). Specifically, we place a period between the DataFrame and the column name (e.g., `loblolly.height`).

Run the code cell below. You will see that the head and tail of the column are displayed in addition to other column information: name, length, and data type. Modify the second code cell below to inspect a column other than height and then run it.

In [15]:
loblolly.height  # inspect the height column in the loblolly DataFrame

0      4.51
1      4.55
2      4.79
3      3.91
4      4.81
      ...  
79    61.62
80    58.49
81    56.81
82    56.43
83    59.49
Name: height, Length: 84, dtype: float64

In [16]:
loblolly.age  # add a column name (other than height) after the period and run the cell to inspect the column

0      3
1      3
2      3
3      3
4      3
      ..
79    25
80    25
81    25
82    25
83    25
Name: age, Length: 84, dtype: int64

#### Indexing

Sometimes it is useful to select specific values within your data. Python uses an indexing system, like the majority of statistical packages, for both rows and columns. In Python, just like R, you must specify the value using the indices, or subscripts as they are sometimes called.

The indices look like this `[r, c]` where *r* is the row value and *c* is the column value. 

The pandas module provides two different methods for indexing and selecting data:

* Selection by position: a suite of methods to access data by integer-based indexing; use `.iloc`
* Selection by label: a collection of methods to access data from a DataFrame by label-based indexing; use `.loc`

##### Integer-based Indexing

The following are considered valid inputs for accessing indexes for integer-based indexing:
* Integers such as `3`, `60`, `99`
* A list object or an array such as `[3, 60, 99]`
* A slice object using integers `3:99`

Recall the `loblolly` DataFrame contains three columns of data. If we wanted to access row 33, we would write the following code:

In [17]:
loblolly.iloc[32]

height     28.66
age        10.00
Seed      309.00
Name: 32, dtype: float64

Why didn't we use the number 33 if we wanted row 33? Remember, Python begins index values at 0, not 1. Thus, the first row has index value 0. For any row you want, subtract the value of 1 to obtain its index value such that `i = r - 1` where *i* is the index value you wish to obtain and *r* is the row you are attempting to access. For row 33, the index value is 32.

What if we want only the first three rows? We could use slicing for this. The code cells below present two possible solutions.

In [18]:
loblolly.iloc[:3]

,height,age,Seed
0,4.51,3,301
1,4.55,3,303
2,4.79,3,305


In [19]:
loblolly.iloc[0:3]

,height,age,Seed
0,4.51,3,301
1,4.55,3,303
2,4.79,3,305


Note, the values here are not inclusive. If we count up the number of index values input, four values should have been returned. Count them: 0, 1, 2, 3. That's four items. Yet, Python only returned 3 rows. Why is this? When using `.iloc` pandas is not inclusive when we perform a slice. For arrays, pandas is inclusive.

Returning to our example, we can use an array as input:

In [20]:
loblolly.iloc[[0, 1, 2]]

,height,age,Seed
0,4.51,3,301
1,4.55,3,303
2,4.79,3,305


Now that we know how to access rows of data, what about columns of data? Simple. For integers, let's select row 3 and column 3. Just like rows, we need to subtract a value of 1 from the column number to obtain the index value:

In [21]:
loblolly.iloc[2, 2]

305

Using slicing as an input, let's select the first three rows and first two columns.

In [ ]:
loblolly.iloc[:3, :2]

,height,age
0,4.51,3
1,4.55,3
2,4.79,3


In [23]:
loblolly.iloc[0:3, 0:2]

,height,age
0,4.51,3
1,4.55,3
2,4.79,3


What about rows 10 through 16 and the last two columns? Sure, we can do that too:

In [24]:
loblolly.iloc[9:16, 1:3]

,age,Seed
9,3,323
10,3,325
11,3,327
12,3,329
13,3,331
14,5,301
15,5,303


Since we are using the technique slicing, we need to add one more value to the last integer in our slices. Instead of using `9:15` we used `9:16`; instead of `1:2` we use `1:3`.

Another important note on slicing. If we would like all rows in the data but only the last two columns, we would do the following:

In [25]:
loblolly.iloc[:, 1:3]

,age,Seed
0,3,301
1,3,303
2,3,305
3,3,307
4,3,309
...,...,...
79,25,323
80,25,325
81,25,327
82,25,329


Moving on to arrays, we can find the first three rows and last two columns:

In [26]:
loblolly.iloc[[0, 1, 2], [1, 2]]

,age,Seed
0,3,301
1,3,303
2,3,305


For a detailed review on integer-based indexing, [check out the pandas user guide](https://pandas.pydata.org/docs/user_guide/indexing.html#indexing-integer).

##### Label-based Indexing

The following are considered valid inputs for accessing indexes for label-based indexing.

* Labels in the form of integers such as `3`, `60`, `90` or names of columns as strings.
* A list or array of labels such as `['height', 'age', 'Seed']` for columns or `['a','b','c']` for rows if your index values are characters and not integers.
* A slice object using labels `'height':'age'`

Using slicing, we can access the columns in our DataFrame like so:


In [27]:
loblolly.loc[:, 'age']

0      3
1      3
2      3
3      3
4      3
      ..
79    25
80    25
81    25
82    25
83    25
Name: age, Length: 84, dtype: int64

This yields all rows of data with just the column `age`. If we would like to obtain all rows for the columns *height* through *age* we could use the following:

In [72]:
loblolly.loc[:, 'height':'age']

,height,age
0,4.51,3
1,4.55,3
2,4.79,3
3,3.91,3
4,4.81,3
...,...,...
79,61.62,25
80,58.49,25
81,56.81,25
82,56.43,25


What about rows? How will slicing work since the index values are numeric in nature? Well, `.loc` will treat them as if they were labels and not integers!

In [29]:
loblolly.loc[0:3, 'height':'age']

,height,age
0,4.51,3
1,4.55,3
2,4.79,3
3,3.91,3


This code pulls the first four records. Note, that this is *inclusive*, unlike `.iloc`. This is because the index values are not treated like integers; they are labels for all intents and purposes.

Now that we understand slicing with labels, how about using arrays? Easy! Remember, we are dealing with labels now, not integers. This means when we select the columns, instead of referencing their index values as integers, we use their actual labels. For the first three rows and last two columns, the result would be the following:

In [30]:
loblolly.loc[[0, 1, 2], ['age', 'Seed']]

,age,Seed
0,3,301
1,3,303
2,3,305


For more details on label-based indexing, [check out the pandas user guide](https://pandas.pydata.org/docs/user_guide/indexing.html#indexing-label).

#### Unique Values

Another useful pandas function is `unique()`. This function identifies the unique values contained within your data. 

If you have categorical data, this can be helpful in determining the various values contained in a variable (in R, the term `factor` is used instead of categorical). Our loblolly DataFrame does not contain categorical data but we will use one of its columns here to demonstrate the code for the `unique()` function. To see a list of unique values in the `age` column, run the following code:

In [31]:
pd.unique(loblolly.age)

array([ 3,  5, 10, 15, 20, 25], dtype=int64)

Now we know that `age` contains six unique values.

#### Row and Column Metadata

Another useful function, or set of functions, provides basic information on row and column size. This may be useful if we do not know the number of records that the data contains or the number of columns. 

Three methods are provided in the code cells below. 

* In the first code cell, `shape` provides information on rows and columns.
* In the second and third code cell, the `len()` function merely assesses the length of the rows and columns.
* In the fourth code cells, `info()` also provides information on rows and columns, including data types.

In [32]:
loblolly.shape

(84, 3)

In [33]:
len(loblolly.index)

84

In [34]:
len(loblolly.columns)

3

In [41]:
loblolly.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84 entries, 0 to 83
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   height  84 non-null     float64
 1   age     84 non-null     int64  
 2   Seed    84 non-null     int64  
dtypes: float64(1), int64(2)
memory usage: 2.1 KB


#### Missing Values

We can also inspect our DataFrame for missing values. The first method returns a TRUE-FALSE value based on whether it is complete; i.e. `TRUE` indicates no missing values whereas `FALSE` indicates missing values. This function is `notnull()`.

```Python
pd.notnull(loblolly)
```
 
To perform the opposite test, use the function `isnull()`. This returns TRUE for missing values and FALSE when no data missing.

```Python
pd.isnull(loblolly)
```
 
An important note: Datetime data types, specifically `datetime64[ns]`, `NaT` represents missing values, whereas `NaN` is typically used in numeric data types. Object data types will use the value provided them. pandas objects are intercompatible between NaT and NaN.

### Adding Rows

An important task we often engage in as we work with data in Python is appending new rows or adding new columns to a DataFrame. pandas provides a simple approach. 

First, create some new rows of data to populate the loblolly data. We are creating these rows as a new DataFrame so that we can easily combine it with the original loblolly DataFrame by using the `concat()` function.

In [36]:
# some new old trees
new_rows = pd.DataFrame({'height': [71.22, 85.05, 68.34], 
                         'age': [30, 30, 30],
                         'Seed': [400, 401, 402]},
                        index = [84, 85, 86]
                       )

Second, we can concatenate our new DataFrame to the existing loblolly DataFrame. Save this as a new DataFrame object named `loblolly_mod_1`. The reason we do this is to create a historical trail. If we need access to the original DataFrame, we cannot overwrite it.

In [39]:
# merge new trees with existing trees data
loblolly_mod_1 = pd.concat([loblolly, new_rows], ignore_index = True)

We can test to see if we successfully added the three new rows by looking at the lengths of the DataFrames.

In [42]:
print(len(loblolly.index.values))  # original DataFrame

print(len(loblolly_mod_1.index.values))  # new DataFrame

84
87


We can also inspect the last three lines of the new DataFrame to see our new rows:

In [43]:
loblolly_mod_1.tail(3)

,height,age,Seed
84,71.22,30,400
85,85.05,30,401
86,68.34,30,402


### Adding Columns 

Okay, we know how to add rows. How about adding columns? pandas has us covered. 

Say we want to create a new column called *growth* containing the annual growth rate of each tree:

In [44]:
new_cols = pd.DataFrame({'diameter': np.random.randint(1, 4, size = 87)})

Using the `concat()` function, we can add the new column of data to the existing DataFrame. By default, `concat()` will stack DataFrames on top of each other (i.e., add new rows). We can tell pandas to place our DataFrames side by side (i.e., add new columns) with the `axis` parameter. Specifically, we set `axis` to `1` as follows:

In [45]:
loblolly_mod_2 = pd.concat([loblolly_mod_1, new_cols], axis = 1)
loblolly_mod_2

,height,age,Seed,diameter
0,4.51,3,301,1
1,4.55,3,303,1
2,4.79,3,305,3
3,3.91,3,307,3
4,4.81,3,309,2
...,...,...,...,...
82,56.43,25,329,3
83,59.49,25,331,2
84,71.22,30,400,1
85,85.05,30,401,1


For more information on this topic, [check out the pandas user guide](http://pandas.pydata.org/pandas-docs/stable/merging.html).

### Removing Columns

Removing columns in Python is a straightforward process. After importing your data into a DataFrame, use the function `drop()` to remove the undesired columns. We will use the newly created `loblolly_mod_2` DataFrame for this section. To start, let's inspect the columns in the data:

In [46]:
loblolly_mod_2.columns

Index(['height', 'age', 'Seed', 'diameter'], dtype='object')

In this example, we are not interested in the columns `Seed` or `diameter`. These columns are the third and fourth columns, respectively. To remove these columns, we would do the following:

In [47]:
loblolly_mod_2.drop(['Seed', 'diameter'], axis = 1)

,height,age
0,4.51,3
1,4.55,3
2,4.79,3
3,3.91,3
4,4.81,3
...,...,...
82,56.43,25
83,59.49,25
84,71.22,30
85,85.05,30


Note that the above code did not store or save our modification to the `loblolly_mod_2` DataFrame. Run the code cell below to check: 

In [50]:
loblolly_mod_2

,height,age,Seed,diameter
0,4.51,3,301,1
1,4.55,3,303,1
2,4.79,3,305,3
3,3.91,3,307,3
4,4.81,3,309,2
...,...,...,...,...
82,56.43,25,329,3
83,59.49,25,331,2
84,71.22,30,400,1
85,85.05,30,401,1


The following code would save the DataFrame with `Seed` and `diameter` dropped:

```Python
loblolly_mod_2 = loblolly_mod_2.drop(['Seed', 'diameter'], axis = 1)
```

As an alternative method, you can use the column index values instead of the actual names:

In [51]:
loblolly_mod_2.drop(loblolly_mod_2.columns[[2, 3]], axis = 1)

,height,age
0,4.51,3
1,4.55,3
2,4.79,3
3,3.91,3
4,4.81,3
...,...,...
82,56.43,25
83,59.49,25
84,71.22,30
85,85.05,30


The first example is a way to remove individual columns of data using the name of the column. If we have a long list of columns to remove, typing out individual column names would become tedious. The second example shows a way in which to remove columns using their index value. 

It should be noted that in Python, the indexing starts at 0 for both columns and rows; in R, indexing values start at 1. Thus, the fourth column in Python has an index value of 3, not 4.

### Renaming Columns

Renaming columns is simple. Python provides many possible ways to change a column header. The first requires that we type in all of the column names, even the ones we are not changing. This can be tedious if we have a lot of columns. If that is the case, the second method is better; also, it is recommended that we remove unwanted columns or create a subset of the data prior to renaming columns. In essence, we are renaming every single column in the DataFrame; you just are not changing some of them.

For the first method, assume we would like to rename the column `diameter` to `trunk_diameter`. Our code for that looks like:

In [52]:
loblolly_mod_2.columns = ['height', 'age', 'Seed', 'trunk_diameter']
loblolly_mod_2.columns

Index(['height', 'age', 'Seed', 'trunk_diameter'], dtype='object')

The second method uses the function `rename()`. This method will only allow the renaming of a single column of data. This avoids having to write out all the names of each column. Rename the column trunk_diameter back to diameter:

In [53]:
loblolly_mod_2.rename(columns = {'trunk_diameter':'diameter'}, inplace = True)
loblolly_mod_2.columns

Index(['height', 'age', 'Seed', 'diameter'], dtype='object')

### Sort Data

We can sort data very easily. We can sort ascending, descending, select multiple columns to sort by, include only certain columns in results, and many other combinations. A simple sort would look like this:

In [54]:
loblolly.sort_values(by = 'height')

,height,age,Seed
13,3.46,3,331
8,3.77,3,321
5,3.88,3,311
3,3.91,3,307
12,3.93,3,329
...,...,...,...
70,60.92,25,301
79,61.62,25,323
74,63.05,25,309
71,63.39,25,303


This sorts the data only on the column *height*. Note that if the data contained missing values, those would be listed toward the bottom of the sort. If we would like the missing values at the top, we would modify the code as follows:

```Python
loblolly.sort_values(by = 'height', na_position = 'first')
```

Sometimes we are only interested in the largest or smallest value within a column of data. Using the functions `nlargest()` and `nsmallest()`, we can obtain those values respectively. For example, say we want the six largest values for `height`:

In [55]:
loblolly.nlargest(6, 'height')

,height,age,Seed
72,64.10,25,305
71,63.39,25,303
74,63.05,25,309
79,61.62,25,323
70,60.92,25,301
77,60.69,25,319


Perhaps we would like the six largest values for both *height* and *age*. The code is not too much of an extension from the previous line:

```Python
loblolly.nlargest(6, ['height', 'age'])
```

Sorting multiple columns is also straight forward. Just add the additional columns in the order that you would like them sorted. If you want to sort by *height* and *age* in that order, you would list *height* first; if you want *age* sorted first, then list *age* first.

```Python
loblolly.sort_values(by = ['height', 'age'])  # sort by height then sort by age

loblolly.sort_values(by = ['age', 'height'])  # sort by age then sort by height
```


### Sampling Data

One basic approach to sampling is basing a subsample on a percentage of the overall sample size. For example, say we would like to sample 10% of the original data and perform an analysis on it. The steps include:
* Determine how many rows is 10% of the data.
* Find out how many rows are in the DataFrame.
* Determine the range of the sample.
* Perform the splitting.

We will import a new dataset from the file diamonds.csv and use it to perform this operation. [This dataset is from R's ggplot2 package](https://ggplot2.tidyverse.org/reference/diamonds.html) and is provided in the data folder of this week's repo.

Some example code for sampling is provided below. Notice the function `astype(int)`. The function `np.round()` returns a number with a decimal point, even when we request zero decimal points. To change the data type from float to an integer, we have to recast the object using `astype(int)`.

In [56]:
# import the data
diamonds = pd.read_csv("diamonds.csv")

# quick inspect
diamonds

,carat,cut,color,clarity,depth,table,price,x,y,z
0,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
1,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
2,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31
3,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63
4,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75
...,...,...,...,...,...,...,...,...,...,...
53935,0.72,Ideal,D,SI1,60.8,57.0,2757,5.75,5.76,3.50
53936,0.72,Good,D,SI1,63.1,55.0,2757,5.69,5.75,3.61
53937,0.70,Very Good,D,SI1,62.8,60.0,2757,5.66,5.68,3.56
53938,0.86,Premium,H,SI2,61.0,58.0,2757,6.15,6.12,3.74


In [57]:
# determine how many rows is 10% of the data
split_num = np.round((len(diamonds.index) * 0.1), 0).astype(int)
split_num

5394

In [58]:
diamonds_subsample = diamonds.sample(n = split_num, replace = False)  # sample by specifying the number of items to return
len(diamonds_subsample.index)

5394

Another option is to skip the steps calculating the number of rows to sample. The function `sample()` provides the argument `frac` that allows you to specify the percentage you would like sampled. This is a quicker method than the previous one shown. Note that Both methods yield a length of 5,394 rows.

In [59]:
diamonds_subsample = diamonds.sample(frac = 0.1, replace = False)  # sample by specifying the fraction of items to return
len(diamonds_subsample.index)

5394

Often we will want to select a subsample based on certain conditions or criteria given the data. First, we read in the data (already done). Second, we convert a variable of interest to a categorical data type. We will use the `cut` column from the diamonds data for this example.

Python uses the data type *object* by default. We can confirm this by using the following code to filter for just *object* data types:

In [60]:
diamonds.select_dtypes(include = ['object'])

,cut,color,clarity
0,Ideal,E,SI2
1,Premium,E,SI1
2,Good,E,VS1
3,Premium,I,VS2
4,Good,J,SI2
...,...,...,...
53935,Ideal,D,SI1
53936,Good,D,SI1
53937,Very Good,D,SI1
53938,Premium,H,SI2


This outputs three columns: `cut`, `color`, and `clarity`. The *object* data type is not the same as *category*. While both behave similarly, they are not equivalent. What is the difference? The reference page for pandas provides this description of *category*:

> The categorical data type is useful in the following cases:
> * A string variable consisting of only a few different values. Converting such a string variable to a categorical variable will save some memory, see [here](https://pandas.pydata.org/pandas-docs/stable/user_guide/categorical.html#categorical-memory).
> * The lexical order of a variable is not the same as the logical order ("one", "two", "three"). By converting to a categorical and specifying an order on the categories, sorting and min/max will use the logical order instead of the lexical order, see [here](https://pandas.pydata.org/pandas-docs/stable/user_guide/categorical.html#categorical-sort).
> * As a signal to other Python libraries that this column should be treated as a categorical variable (e.g. to use suitable statistical methods or plot types).

If you have categorical data in a dataset, you should always convert it. In this case, we have three columns with categorical data that should be converted, but we will focus on one column, `cut`, to keep things simple.

Okay, let's convert the data type for `cut` from *object* to *category*:

In [61]:
diamonds['cut'] = diamonds['cut'].astype('category')

*cut* contains five unique values as shown below

In [62]:
diamonds.cut.unique()

['Ideal', 'Premium', 'Good', 'Very Good', 'Fair']
Categories (5, object): ['Fair', 'Good', 'Ideal', 'Premium', 'Very Good']

Let's assume we would like to perform an analysis only on diamonds with an *Ideal* cut.

In [63]:
diamonds[diamonds.cut == 'Ideal']

,carat,cut,color,clarity,depth,table,price,x,y,z
0,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
11,0.23,Ideal,J,VS1,62.8,56.0,340,3.93,3.90,2.46
13,0.31,Ideal,J,SI2,62.2,54.0,344,4.35,4.37,2.71
16,0.30,Ideal,I,SI2,62.0,54.0,348,4.31,4.34,2.68
39,0.33,Ideal,I,SI2,61.8,55.0,403,4.49,4.51,2.78
...,...,...,...,...,...,...,...,...,...,...
53925,0.79,Ideal,I,SI1,61.6,56.0,2756,5.95,5.97,3.67
53926,0.71,Ideal,E,SI1,61.9,56.0,2756,5.71,5.73,3.54
53929,0.71,Ideal,G,VS1,61.4,56.0,2756,5.76,5.73,3.53
53935,0.72,Ideal,D,SI1,60.8,57.0,2757,5.75,5.76,3.50


After perusing the *Ideal* diamond data, we realize that we only want data with a price value less than or equal to 400. This is another simple process. We just append additional conditions using the symbol `&`.

In [64]:
diamonds[(diamonds.cut == 'Ideal') & (diamonds.price <= 400)]

,carat,cut,color,clarity,depth,table,price,x,y,z
0,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
11,0.23,Ideal,J,VS1,62.8,56.0,340,3.93,3.90,2.46
13,0.31,Ideal,J,SI2,62.2,54.0,344,4.35,4.37,2.71
16,0.30,Ideal,I,SI2,62.0,54.0,348,4.31,4.34,2.68
28262,0.25,Ideal,H,SI1,62.8,54.0,357,4.05,4.07,2.55
...,...,...,...,...,...,...,...,...,...,...
47308,0.26,Ideal,G,VS2,62.3,54.0,397,4.09,4.10,2.55
47309,0.26,Ideal,H,SI1,62.2,54.0,397,4.10,4.13,2.56
50610,0.27,Ideal,H,VS2,60.5,57.0,397,4.17,4.19,2.53
50617,0.31,Ideal,J,VS2,61.8,54.1,400,4.35,4.38,2.70


What if we want to find data for diamonds that have an ideal or premium cut? 

We would type in both conditions and separate them using the OR operator, which is `|`:

In [65]:
diamonds[(diamonds.cut == 'Ideal') | (diamonds.cut == 'Premium')]

,carat,cut,color,clarity,depth,table,price,x,y,z
0,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
1,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
3,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63
11,0.23,Ideal,J,VS1,62.8,56.0,340,3.93,3.90,2.46
12,0.22,Premium,F,SI1,60.4,61.0,342,3.88,3.84,2.33
...,...,...,...,...,...,...,...,...,...,...
53931,0.71,Premium,F,SI1,59.8,62.0,2756,5.74,5.73,3.43
53934,0.72,Premium,D,SI1,62.7,59.0,2757,5.69,5.73,3.58
53935,0.72,Ideal,D,SI1,60.8,57.0,2757,5.75,5.76,3.50
53938,0.86,Premium,H,SI2,61.0,58.0,2757,6.15,6.12,3.74


You may wish to create subsamples to perform techniques requiring training, testing, validation data or even k-fold cross validation data. For more information on using the scikit-learn and its functions, please review this webpage for [scikit-learn](http://scikit-learn.org/stable/modules/cross_validation.html#k-fold).

In [66]:
kf = KFold(n_splits = 2)  # K-Fold cross-validator with 2 folds (the default is 5)

for train, test in kf.split(diamonds):
    print("%s %s" % (train, test))

[26970 26971 26972 ... 53937 53938 53939] [    0     1     2 ... 26967 26968 26969]
[    0     1     2 ... 26967 26968 26969] [26970 26971 26972 ... 53937 53938 53939]


In [67]:
diamonds.iloc[train]

,carat,cut,color,clarity,depth,table,price,x,y,z
0,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
1,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
2,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31
3,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63
4,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75
...,...,...,...,...,...,...,...,...,...,...
26965,1.07,Ideal,D,IF,60.9,54.0,17042,6.66,6.73,4.08
26966,2.13,Premium,F,SI2,61.5,58.0,17045,8.28,8.33,5.11
26967,2.07,Ideal,I,VS2,60.3,59.0,17045,8.23,8.28,4.98
26968,1.71,Ideal,G,VS1,61.4,57.0,17049,7.65,7.72,4.72


In [68]:
diamonds.iloc[test]

,carat,cut,color,clarity,depth,table,price,x,y,z
26970,1.60,Ideal,G,VS1,62.0,54.0,17050,7.53,7.50,4.60
26971,2.27,Good,J,VS1,59.3,61.0,17051,8.47,8.55,5.05
26972,2.09,Ideal,G,VS2,61.8,57.0,17051,8.24,8.17,5.07
26973,1.71,Very Good,F,VS2,58.5,61.0,17052,7.88,7.79,4.58
26974,2.32,Premium,J,VS2,62.4,57.0,17053,8.48,8.40,5.27
...,...,...,...,...,...,...,...,...,...,...
53935,0.72,Ideal,D,SI1,60.8,57.0,2757,5.75,5.76,3.50
53936,0.72,Good,D,SI1,63.1,55.0,2757,5.69,5.75,3.61
53937,0.70,Very Good,D,SI1,62.8,60.0,2757,5.66,5.68,3.56
53938,0.86,Premium,H,SI2,61.0,58.0,2757,6.15,6.12,3.74


### Handling Dates and Times

All modern statistical packages provide functions that perform mathematical operations and dates and times. 

Say we have data on employee work hours and we need to calculate pay for hourly employees. For each day over a five-day span we have the time the employee clocked in and the time the employee clocked out. We need to calculate the total number of hours the employee worked by using one of two methods: 1) we convert the date-time values into separate numerical values such as hours, minutes, and seconds and sum up the values or 2) use a date-time function that will automatically convert for us and provide you the total hours.

The second option is the obvious choice as it requires minimal computational skills on our part. One of the downsides to most statistical packages is that they do not convert date-time values into date-time objects. That is, date-time values are read as categorical values made up of character strings; we cannot perform math on character strings. Converting date-time values in any statistical program requires work. 

Let's go over how to convert date-time values from *object* data types to *date-time* data types. We will use the data in the tennis_serve_time.csv file for this example. [This dataset is from the R fivethirtyeight package](https://vincentarelbundock.github.io/Rdatasets/doc/fivethirtyeight/tennis_serve_time.html) and is provided in the data folder of this week's repo.

In [69]:
tennis = pd.read_csv('tennis_serve_time.csv')  # read in tennis serve time data
tennis.dtypes  # check data types

rownames        int64
server         object
sec_between     int64
opponent       object
game_score     object
set             int64
game           object
date           object
dtype: object

The process of converting to a date-time object is simpler in Python than it is in R because the pandas library provides powerful tools. 

We will focus on the *date* column for this example. Looking at the data itself reveals its string character formatting:

In [70]:
tennis.date.head()

0    2015-05-28
1    2015-05-28
2    2015-05-28
3    2015-05-28
4    2015-05-28
Name: date, dtype: object

To convert the column to a date-time format, we can use the `pd.to_datetime()` function. The data type is now `datetime64[ns]` instead of `object`.

In [71]:
tennis['date'] = pd.to_datetime(tennis['date'])
tennis.date.head()

0   2015-05-28
1   2015-05-28
2   2015-05-28
3   2015-05-28
4   2015-05-28
Name: date, dtype: datetime64[ns]

---

That wraps up this week's tutorial! Review this content as necessary.

## Supplemental Materials: Data

You may have noticed there are some datasets in the data folder that we did not interact with in the tutorial. I have included these data within the Week 7 GitHub repo in case you would like to practice and apply the methods described here to other datasets. Like the other datasets in this tutorial, most of these datasets are from R packages. The only exceptions are the data that you will use in your lab assignments:

* github_teams.csv will be used in this week's *instructor-led* lab.
  * Source: me (Olivia B. Newton)
* CaliforniaHospitalData.csv and CaliforniaHospitalData_Personnel.txt will be used in this week's *independent* lab.
  * Source: UM COB MIS professor Dr. Hammer

If you are interested in practicing with more data beyond what is provided in this GitHub repo, [you can find and download a large variety of datasets associated with R packages on this handy website](https://vincentarelbundock.github.io/Rdatasets/index.html). 